In [197]:
import gensim.downloader as api

# Load pre-trained GloVe model
model = api.load("glove-wiki-gigaword-100")

In [213]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd


#df = pd.read_csv("C:/Users/chiar/Downloads/final500 (2).csv")
df = pd.read_csv("C:/Users/chiar/Downloads/final10000.csv")
#df = df.dropna(subset=['Description'])
df =  df.dropna(subset=['review/text'])            
df = df.dropna(subset=['num_pages'])
df = df.drop(columns=['nan'])
df = df.drop(columns=['Unnamed: 0'])
df = df.drop(df[df["review/text"] == "['nan']"].index)
df = df.drop(columns=['Description'])
#df = df.drop(columns=['review/text'])
#df.to_csv("final_1365books.csv")

df

,Title,Author,Genre,num_pages,review/text
3,And Then There Were None,Agatha Christie,Mystery,264.0,"['book', 'one', 'ten', 'read', 'mystery', 'kil..."
6,"The Lion, the Witch and the Wardrobe",C. S. Lewis,"Fantasy, Children's fiction",206.0,"['lewis', 'world', 'witch', 'fantasy', 'wardro..."
10,Harry Potter and the Chamber of Secrets,J. K. Rowling,Fantasy,352.0,"['book', 'potter', 'harry', 'fiction', 'scienc..."
17,The Catcher in the Rye,J. D. Salinger,Coming-of-age,234.0,"['amp', 'book', 'reading', 'read', 'biography'..."
22,Lolita,Vladimir Nabokov,Novel,335.0,"['lolita', 'humbert', 'language', 'nabokov', '..."
...,...,...,...,...,...
9431,The Human Mind,Robert Winston,"Nonfiction,Psychology,Science,Biology,Neurosci...",528.0,"['brain', 'winston', 'book', 'information', 's..."
9433,This Little Chick,John Lawrence,"Childrens,Picture Books,Animals,Childrens,Stor...",32.0,"['little', 'quot', 'chick', 'animal', 'reading..."
9452,View with a Grain of Sand: Selected Poems,"WisÅ‚awa Szymborska,StanisÅ‚aw BaraÅ„czak,Clar...","Poetry,Cultural,Poland,European Literature,Pol...",214.0,"['poems', 'read', 'poetry', 'really', 'book', ..."
9485,Rogue Male,Geoffrey Household,"Fiction,Thriller,Mystery,Crime,Mystery,Adventu...",182.0,"['book', 'country', 'british', 'tale', 'like',..."


In [214]:

img = pd.read_csv("C:/Users/chiar/Downloads/moods (1).csv")
img["image"] = [0, 1, 2, 3, 4, 5, 6, 7, 8]

#getting the summary vector for the books' descriptions
for index, row in df.iterrows():
    review_text = row['review/text']
    if review_text != str(['nan']):
        review_text = review_text.strip("[] ' ' \"  ")
        review_text = review_text.replace("'", "")
        review_text = review_text.replace(" ", "")
        li = list(review_text.split(","))
        li = [word for word in li if word in model]
        summary_vector = sum(model[word] for word in li) / len(li)
        df.at[index, 'review/text'] = summary_vector
df 

#getting the summary vector for the images descriptions
for index, row in img.iterrows():
    review_text = row['vector']
    if review_text != str(['nan']):
        li = list(review_text.split(","))
        li = [word for word in li if word in model]
        summary_vector = sum(model[word] for word in li) / len(li)
        img.at[index, 'vector'] = summary_vector
img

def create_similarity_matrix(book_description_vectors, mood_vectors):
    similarity_matrix = np.zeros((len(book_description_vectors), len(mood_vectors)))
    for i, book_vector in enumerate(book_description_vectors):
        for j, mood_vector in enumerate(mood_vectors):
            if isinstance(book_vector, str) and book_vector == "['nan']":
                similarity_matrix[i, j] = 0
            else:
                #this check now becomes useless because we removed those earlier
                if np.isnan(book_vector).any() or np.isnan(mood_vector).any():
                    similarity_matrix[i, j] = 0
                else:
                    similarity_score = cosine_similarity([book_vector], [mood_vector])[0][0]
                    similarity_matrix[i, j] = similarity_score
    return similarity_matrix

mood_vectors = img['vector']
book_vectors = df['review/text']
book_titles = df['Title'].tolist()
similarity_matrix = create_similarity_matrix(book_vectors, mood_vectors)

matrix = pd.DataFrame(similarity_matrix, index=book_titles, columns=["happy", "sad", "misterious", "spring", "adventurous", "thought-provoking", "romance", "cozy", "fun"])
matrix

non_zero_rows = matrix[(matrix > 0.0000000).any(axis=1)]
#non_zero_rows.to_csv("mood_matrix.csv")

non_zero_rows

,happy,sad,misterious,spring,adventurous,thought-provoking,romance,cozy,fun
And Then There Were None,0.708539,0.460718,0.367791,0.415213,0.475417,0.542501,0.718039,0.184678,0.620200
"The Lion, the Witch and the Wardrobe",0.593243,0.403501,0.364485,0.361698,0.603045,0.509143,0.722981,0.175687,0.608017
Harry Potter and the Chamber of Secrets,0.661139,0.474719,0.517526,0.359339,0.617930,0.562555,0.772843,0.243108,0.701532
The Catcher in the Rye,0.679209,0.488919,0.411122,0.377003,0.483698,0.631479,0.753063,0.204740,0.633474
Lolita,0.651534,0.456875,0.409290,0.351022,0.495811,0.593820,0.739397,0.182729,0.571301
...,...,...,...,...,...,...,...,...,...
The Human Mind,0.582039,0.359734,0.391833,0.341313,0.435736,0.561223,0.576303,0.181041,0.539014
This Little Chick,0.763816,0.511183,0.381383,0.454744,0.469397,0.535775,0.747039,0.275895,0.692513
View with a Grain of Sand: Selected Poems,0.697610,0.460941,0.401551,0.372800,0.474141,0.606666,0.729281,0.163858,0.629822
Rogue Male,0.627084,0.436670,0.476129,0.332246,0.555670,0.584509,0.700178,0.235087,0.614593


## option 2 -> process stopped, we will stick with the 1365 words

In [217]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


#df = pd.read_csv("C:/Users/chiar/Downloads/final500 (2).csv")
df = pd.read_csv("C:/Users/chiar/Downloads/final10000.csv")
df =  df.dropna(subset=['review/text'])            
df = df.dropna(subset=['num_pages'])
df = df.drop(columns=['nan'])
df = df.drop(columns=['Unnamed: 0'])
#df = df.drop(df[df["review/text"] == "['nan']"].index)
#df = df.drop(columns=['Description'])
#df = df.drop(columns=['review/text'])
#df.to_csv("final_1365books.csv")

df

,Title,Author,Genre,Description,num_pages,review/text
0,A Tale of Two Cities,Charles Dickens,Historical fiction,Edited with an Introduction and Notes by Richa...,489.0,['nan']
3,And Then There Were None,Agatha Christie,Mystery,NaN,264.0,"['book', 'one', 'ten', 'read', 'mystery', 'kil..."
6,"The Lion, the Witch and the Wardrobe",C. S. Lewis,"Fantasy, Children's fiction",NaN,206.0,"['lewis', 'world', 'witch', 'fantasy', 'wardro..."
9,The Da Vinci Code,Dan Brown,Mystery thriller,"While in Paris on business, Harvard symbologis...",489.0,['nan']
10,Harry Potter and the Chamber of Secrets,J. K. Rowling,Fantasy,When the Chamber of Secrets is opened again at...,352.0,"['book', 'potter', 'harry', 'fiction', 'scienc..."
...,...,...,...,...,...,...
9494,"Yurara, Vol. 1",Chika Shiomi,"Sequential Art,Manga,Fantasy,Sequential Art,Gr...",NaN,200.0,['nan']
9495,"Yurara, Vol. 2",Chika Shiomi,"Sequential Art,Manga,Fantasy,Romance,Sequentia...",NaN,192.0,['nan']
9496,Finn,Jon Clinch,"Fiction,Historical,Historical Fiction,Historic...",NaN,287.0,['nan']
9497,"God, Justice, and Society: Aspects of Law and ...",Jonathan Burnside,"Law,Religion,Theology",NaN,542.0,['nan']


In [ ]:
img = pd.read_csv("C:/Users/chiar/Downloads/moods (1).csv")
img["image"] = [0, 1, 2, 3, 4, 5, 6, 7, 8]

#getting the summary vector for the books' descriptions
for index, row in df.iterrows():
    review_text = row['review/text']
    if review_text != str(['nan']):
        review_text = review_text.strip("[] ' ' \"  ")
        review_text = review_text.replace("'", "")
        review_text = review_text.replace(" ", "")
        li = list(review_text.split(","))
        li = [word for word in li if word in model]
        summary_vector = sum(model[word] for word in li) / len(li)
        df.at[index, 'review/text'] = summary_vector
df 

#getting the summary vector for the images descriptions
for index, row in img.iterrows():
    review_text = row['vector']
    if review_text != str(['nan']):
        li = list(review_text.split(","))
        li = [word for word in li if word in model]
        summary_vector = sum(model[word] for word in li) / len(li)
        img.at[index, 'vector'] = summary_vector

img

def create_similarity_matrix(book_description_vectors, mood_vectors):
    similarity_matrix = np.zeros((len(book_description_vectors), len(mood_vectors)))
    for i, book_vector in enumerate(book_description_vectors):
        for j, mood_vector in enumerate(mood_vectors):
            if isinstance(book_vector, str) and book_vector == "['nan']":
                similarity_matrix[i, j] = 0
            else:
                #this check now becomes useless because we removed those earlier
                if np.isnan(book_vector).any() or np.isnan(mood_vector).any():
                    similarity_matrix[i, j] = 0
                else:
                    similarity_score = cosine_similarity([book_vector], [mood_vector])[0][0]
                    similarity_matrix[i, j] = similarity_score
    return similarity_matrix

mood_vectors = img['vector']
book_vectors = df['review/text']
book_titles = df['Title'].tolist()
similarity_matrix = create_similarity_matrix(book_vectors, mood_vectors)

matrix = pd.DataFrame(similarity_matrix, index=book_titles, columns=["happy", "sad", "misterious", "spring", "adventurous", "thought-provoking", "romance", "cozy", "fun"])
matrix

non_zero_rows = matrix[(matrix > 0.0000000).any(axis=1)]
#non_zero_rows.to_csv("mood_matrix_reviews.csv")

non_zero_rows